## Read a day of rhd/rhs recordings into a .kwd file

# this will eventually go in pipefinch.neural.convert

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from intan2kwik import kwd as kwd

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.info('Logger set')

2018-12-17 11:16:33,390 - root - INFO - Logger set


In [2]:
exp_base = os.path.abspath('/Users/zeke/experiment/')

file_sys = {'exp_base': os.path.abspath('/Users/zeke/experiment/')}

sess_par = {'bird': 'o3g3',
           'sess': 'awake_mix_test',
           'rec': 0}

file_sys_defaults = {'exp_base': os.path.abspath('/data/experiment')}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)
    f_n['ss'] = os.path.join()

In [3]:
raw_folder = os.path.join(exp_base, 'raw_data', sess_par['bird'], 'neural', sess_par['sess'])
ss_folder = os.path.join(exp_base, 'raw_data', sess_par['bird'], 'ss_data', sess_par['sess'])
logger.info('Raw folder {}'.format(raw_folder))
all_rhd_list = glob.glob(os.path.join(raw_folder, '*.rhd'))
all_rhd_list.sort()

2018-12-17 11:16:33,406 - root - INFO - Raw folder /Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test


In [25]:
raw_folder

'/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test'

In [4]:
all_rhd_list[:20]

['/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_143446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_185537.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_190537.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_191538.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_192538.rhd']

### Sort the files of the session into recording epochs

In [5]:
import datetime

def datetime_from_filename(filename: str, base_name='raw_intan'):
    datetime_str = filename.split('.')[0].split(base_name + '_')[-1].replace('_', '')
    datetime_tuple = tuple([int(datetime_str[i:i+2]) for i in range(0,12,2)])
    t_stamp = datetime.datetime(*datetime_tuple)
    
    # now date is yymmdd, time is hhmmss
    # turn into datetime object
    return t_stamp

def get_rec_breaks(rhd_pd: pd.DataFrame):
    rhd_pd['t_diff'] = rhd_pd['t_stamp'].diff()
    rhd_pd['rec_break'] = rhd_pd['t_diff'].apply(lambda x: True if x.seconds%60>0 else False)
    break_indices = np.hstack([rhd_pd[rhd_pd['rec_break']].index.values, [rhd_pd.index.values[-1], 0]])
    break_indices.sort()
    rec_segments = np.vstack([break_indices[:-1], break_indices[1:]]).T

    for i_rec, segment_edges in enumerate(rec_segments):
        rhd_pd.loc[segment_edges[0]:segment_edges[1], 'rec'] = i_rec
    rhd_pd['rec'] = rhd_pd['rec'].astype(np.int)
    return rhd_pd
    
def get_rhd_pd(raw_folder: str):
    all_rhd_list = glob.glob(os.path.join(raw_folder, '*.rhd'))
    all_rhd_list.sort()
    all_rhd_pd = pd.DataFrame(all_rhd_list, columns=['path'])
    all_rhd_pd['f_name'] = all_rhd_pd['path'].apply(lambda x: os.path.split(x)[1])
    all_rhd_pd['t_stamp'] = all_rhd_pd['f_name'].apply(lambda x: datetime_from_filename(x))
    #all_rhd_pd.loc[-1:, 'rec_break'] = True
    all_rhd_pd = get_rec_breaks(all_rhd_pd)
    return all_rhd_pd

rhd_pd = get_rhd_pd(raw_folder)
rhd_pd.head(20)

,path,f_name,t_stamp,t_diff,rec_break,rec
0,/Users/zeke/experiment/raw_data/o3g3/neural/aw...,raw_intan_180908_143446.rhd,0018-09-08 14:34:46,NaT,False,0
1,/Users/zeke/experiment/raw_data/o3g3/neural/aw...,raw_intan_180908_185537.rhd,0018-09-08 18:55:37,04:20:51,True,1
2,/Users/zeke/experiment/raw_data/o3g3/neural/aw...,raw_intan_180908_190537.rhd,0018-09-08 19:05:37,00:10:00,False,1
3,/Users/zeke/experiment/raw_data/o3g3/neural/aw...,raw_intan_180908_191538.rhd,0018-09-08 19:15:38,00:10:01,True,2
4,/Users/zeke/experiment/raw_data/o3g3/neural/aw...,raw_intan_180908_192538.rhd,0018-09-08 19:25:38,00:10:00,False,2


### Make one .kwd file for the session
 - Each rec goes into one recording

In [6]:
np.unique(rhd_pd.rec)

array([0, 1, 2])

In [7]:
list(rhd_pd[rhd_pd['rec']==0]['path'])

['/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_143446.rhd']

In [64]:
reload(kwd)
kwd_file_name = 'experiment2.kwd'
kwd_file_path = os.path.join(ss_folder, kwd_file_name)
os.makedirs(ss_folder, exist_ok=True)

#kwd.intan_to_kwd_multirec(rhd_pd, kwd_file_path)
first_header, all_rhx_pd = kwd.intan_to_kwd(raw_folder, kwd_file_path)

2018-12-17 23:55:03,378 - intan2kwik.kwd - INFO - reading intan chans data across all of rec /Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test
2018-12-17 23:55:03,386 - intan2kwik.kwd - INFO - Found 5 .rhd files split in 3 recordings
2018-12-17 23:55:03,386 - intan2kwik.kwd - INFO - dest file: /Users/zeke/experiment/raw_data/o3g3/ss_data/awake_mix_test/experiment2.kwd
2018-12-17 23:55:03,387 - intan2kwik.kwd - INFO - tmp path /var/folders/gk/7z94s6yj40n4ggn5dl62xhbc0000gn/T/tmpa66rqcpi/experiment2.kwd


2018-12-17 23:57:48,484 - intan2kwik.kwd - INFO - moving back to /Users/zeke/experiment/raw_data/o3g3/ss_data/awake_mix_test/experiment2.kwd
2018-12-17 23:57:48,495 - intan2kwik.kwd - INFO - removing temp file


In [9]:
import h5py
arr = np.array(list(rhd_pd['path']), dtype=h5py.special_dtype(vlen=str))
arr

array(['/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_143446.rhd',
       '/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_185537.rhd',
       '/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_190537.rhd',
       '/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_191538.rhd',
       '/Users/zeke/experiment/raw_data/o3g3/neural/awake_mix_test/raw_intan_180908_192538.rhd'],
      dtype=object)

In [17]:
import h5py 

with h5py.File(kwd_file_path, 'a') as g:
    
    dg = g['/recordings/0']
    dg['application_data'].create_dataset('tuvieja115', data=arr, dtype=arr.dtype)

In [16]:
import h5py 

with h5py.File(kwd_file_path, 'r') as g:
    
    dg = g['/recordings/0']
    #ds = np.array(dg['application_data/t_stamp_end'])
    dgn = dg.name
    
dgn.split('/')[-1]

'0'

In [29]:
h5py.__version__

'2.8.0'

In [28]:
import tempfile
tf = tempfile.TemporaryFile()
h5_temp = h5py.File(tf)

TypeError: expected str, bytes or os.PathLike object, not _io.BufferedRandom

In [45]:
all_rhx_pd.loc[0, 't_stamp']

datetime.datetime(18, 9, 8, 14, 34, 46)

In [47]:
all_rhx_pd.loc[0, 't_stamp'].strftime("%y-%m-%d %H:%M:%S")

'18-09-08 14:34:46'

In [42]:
dtt.strftime??

Docstring: format -> strftime() style string.
Type:      builtin_function_or_method


In [58]:
rec_rhx_pd = all_rhx_pd[all_rhx_pd['rec'] == 1]


In [63]:
rec_rhx_pd['t_stamp'].values[0]

datetime.datetime(18, 9, 8, 18, 55, 37)